In [11]:
#Edits made by Paddington - Faerlina  Sept 25, 2021
#Added:
#Mana pool
#Mana regen
#Mana potions
#Toggleable abilities
#Insect Swarm
#Starfall downranking
#Individual Spell damage and percentage - Commented out currently for debugging
#Dark Iron Smoking Pipe
#Added trinket internal cooldown in case of using 2 on-use trinkets
#Fixed problem where adding loops in main cell caused many loops with 0 damage
#Changed output to average dps which seems more accurate.



import numpy
import pandas as pd
import math #Paddington - For sqrt in spirit calculation in future


# Define here your stats and environment
# Stats :
intel = 430
crit_score = 107
hit_score = 174 # 113
spellpower = 882
haste = 0
spirit = 233

#Paddington - Added toggling of spells
casting_SF = True
casting_wrath = False
SF_rank = 6
is_MF = False
is_IS = False

is_csd = False # Chaotic Skyfire Diamond equiped
is_spellstrike = False
is_spellfire = False
is_motw = True
is_divine_spirit = True
is_arcane_brilliance = True #40 intellect +14 motw
is_totem_of_wrath = True
is_wrath_of_air = True
is_draenei_in_group = False
curse_of_elements = 1.0 #1.10
is_blessing_kings = False # 10% stats increased
is_crusader = False
is_sham_4_piece = True
is_flask = False
is_food = False
is_wizard_oil = False
#TODO Paddington - Add mana oil
is_mana_oil = False
is_twilight_owl = False
is_eye_of_night = False
is_T5 = False
is_T6_2 = False
is_T6_4 = False
is_drums = False
is_lust = False
lust_count = 1
lust_when = 100 # Percentage at which you want to receive the first bloodlust. 100 means the first lust goes off on engage. 

#TODO Paddington - Add mana regeneration from cooldowns
is_manapot = True
is_innervate = True
is_mana_totem = True
is_spriest = True
misery = 1.05


## Update stats
if is_motw:
    intel = intel + 18
    spirit = spirit + 18
if is_arcane_brilliance:
    intel = intel + 40 
if is_divine_spirit:
    spirit = spirit + 50
if is_totem_of_wrath:
    hit_score = hit_score + (3 * 12.6)
    crit_score = crit_score + (3 * 22.1) 
if is_draenei_in_group:
    hit_score = hit_score + 12.6 
if is_blessing_kings:
    intel = intel * 1.1
    spirit = spirit * 1.1
if is_crusader:
    crit_score = crit_score + (3 * 22.1) 
if is_wrath_of_air:
    spellpower = spellpower + 101
if is_sham_4_piece:
    spellpower = spellpower + 20
if is_flask:
    spellpower = spellpower + 80
if is_food:
    spellpower = spellpower + 20
if is_wizard_oil: 
    spellpower = spellpower + 36
    crit_score = crit_score + 14
if is_twilight_owl:
    crit_score = crit_score + (2 * 22.1)
if is_eye_of_night:
    spellpower = spellpower + 34    

# Paddington - Added mana pool   
mana = 2370 + intel*15


# Functions
def logfunc(s):
    is_log_on = False
    if is_log_on:
        print(s)

def compute_dps(mana, intel, crit_score, hit_score, spellpower, haste, is_csd,
                is_spellstrike, is_spellfire, is_T5, is_T6_2, is_T6_4, is_manapot, is_MF,
                is_IS, SF_rank, casting_wrath, casting_SF):
    #print("Provided values : " + " int " + str(intel) + " crit " +  str(crit_score) + " hit " +  str(hit_score) + " sp " +  str(spellpower)+ " spirit " + str(spirit))
    
    # Special Trinkets
    eye_of_mag = False # +54 SP + Grants 170 increased spell damage for 10 sec when one of your spells is resisted.
    #Paddington - Added Dark Iron Smoking Pipe
    dark_iron_smoking_pipe = True # + 43 + Use: Increase damage and healing done by magucal spells and effects by up to 155 for 20 sec. (2 Min Cooldown)
    silver_crescent = False # 43 SP + Use: Increases damage and healing done by magical spells and effects by up to 155 for 20 sec. (2 Min Cooldown)
    scryer_gem = True # 32 Hit rating + Use: Increases spell damage by up to 150 and healing by up to 280 for 15 sec. (1 Min, 30 Sec Cooldown)
    quagmirran = False # 37 SP + Equip: Your harmful spells have a chance to increase your spell haste rating by 320 for 6 secs. (Proc chance: 10%, 45s cooldown)
    essence_sapphi = False #  Use: Increases damage and healing done by magical spells and effects by up to 130 for 20 sec. (2 Min Cooldown)
    illidari_vengeance = False # +26 Crit . Use: Increases spell damage done by up to 120 and healing done by up to 220 for 15 sec. (1 Min, 30 Sec Cooldown)
    lightening_capacitor = False # Equip: You gain an Electrical Charge each time you cause a damaging spell critical strike.  When you reach 3 Electrical Charges, -> 750 dmg on average (2.5s ICD for charges, concern with wrath spam only)
    xiris_gift = False #  +32 crit / Use: Increases spell damage by up to 150 and healing by up to 280 for 15 sec. (1 Min, 30 Sec Cooldown)
    sextant_of_unstable_currents = False # +40 crit, Chance on critical strike to increase your spellpower by 190 for 15 seconds (20% proc chance, 45 second icd)
    ashtongue_talisman = False # Starfire has a 25% chance to grant you 150 spellpower for 8 seconds (25% proc chance, no icd)
    skull = False # +55 spellpower, +25 hit, 175 haste for 20 seconds on a 120 second cooldown. 
    
    # Talents
    balance_of_power = 4 # +4% Hit
    focused_starlight = 4 # +4% crit for SF and Wrath
    moonkin_form = 5 # +5% Crit
    improved_mf = 10 # +10% Moonfire crit 
    starlight_wrath = True # reduce cast time by 0.5s
    vengeance = True # +100% Crit damange
    lunar_guidance = True # Spellpower bonus = 24% of total intel
    moonfury = 1.1 # +10% damage
    wrath_of_cenarius = 1.2 # +20% Spellpower for SF | +10% SpellPower for Wrath
    fight_length = 155 # in seconds
    is_activable_trinket = True

    # Sets bonuses
    spellfire = is_spellfire # SP bonus = +7% of total intellect
    spellstrike = is_spellstrike # 5% chance to have +92sp for 10s - No ICD
    windhawk = False # 8MP/5 KEK

    # Meta GEM - Chaotic Skyfire Diamond
    csd_equiped = is_csd
    
    #Paddington - Added mana regen
    #Mana Regen
    casting_mp5 = 75
    mp5_tick= 5
    

    # Two kinds of trinkets
    is_trinket_activable = False
    is_trinket_triggered = True
    # Apply stats modifications
    if sextant_of_unstable_currents:
        crit_score = crit_score + 40
        # trinket_duration = 15 # seconds
        sextant_icd = 45 #seconds
        # spellpower_trinket_bonus = 190
        is_trinket_triggered = True
        is_trinket_activable = False
        
    if eye_of_mag:
        spellpower = spellpower + 54
        # trinket_duration = 10 # seconds
        # trinket_cd = 0 # seconds
        # spellpower_trinket_bonus = 170
        is_trinket_triggered = True
        is_trinket_activable = False
            
    if xiris_gift:
        crit_score = crit_score + 32
        spellpower_trinket_bonus = 150
        trinket_duration = 15 # seconds
        trinket_cd = 90 # seconds
        on_use_icd = 15 # seconds
        is_trinket_activable = True 
    #Paddington - Added Dark Iron Smoking Pipe
    if dark_iron_smoking_pipe:
        spellpower = spellpower + 43
        spellpower_trinket_bonus = 155
        trinket_duration = 20 # seconds
        trinket_cd = 120 # seconds
        on_use_icd = 15 # seconds
        is_trinket_activable = True 
        
    if silver_crescent:
        spellpower = spellpower + 43
        spellpower_trinket_bonus = 155
        trinket_duration = 20 # seconds
        trinket_cd = 120 # seconds
        on_use_icd = 15 # seconds
        is_trinket_activable = True 
    
    if essence_sapphi:
        spellpower = spellpower + 40
        spellpower_trinket_bonus = 130
        trinket_duration = 20 # seconds
        trinket_cd = 120 # seconds
        is_trinket_activable = True     

    if scryer_gem:
        #hit_score = hit_score + 32    #Paddington - removed hit because it is already added when manually entering hit rating
        is_trinket_activable = True
        spellpower_trinket_bonus = 150
        trinket_duration = 15 # seconds
        on_use_icd = 15 # seconds
        trinket_cd = 90 # seconds
    
    if illidari_vengeance:
        crit_score = crit_score + 26
        spellpower_trinket_bonus = 120
        trinket_duration = 15 # seconds
        trinket_cd = 90 # seconds
        on_use_icd = 15 # seconds
        is_trinket_activable = True 
    
    if quagmirran:
        spellpower = spellpower + 37
        # haste_trinket_bonus = 320 / 15.77
        # trinket_duration = 6 # seconds
        trinket_icd = 45 # seconds
        # trinket_activation_chance = 0.1 # 10%
    
    if skull:
        spellpower = spellpower + 55
        #hit_score = hit_score + 25   #Paddington - removed hit because it is already added when manually entering hit rating
        trinket_duration = 20 #seconds
        trinket_cd = 120 #seconds
        on_use_icd = 15 # seconds
        
    # Translating stats to %
    # At level 70, 22.1 Spell Critical Strike Rating increases your chance to land a Critical Strike with a Spell by 1%
    # At level 70, 12.6 Spell Hit Rating increases your chance to Hit with Spells by 1%. Hit cap is 202 FLAT (not including talents & buffs).
    # Druids receive 1% Spell Critical Strike chance for every 79.4 points of intellect.

    
    FF_mana_cost = 145
    
    #Paddington - changed spell damages to refelct in game damage.
    # Moonfire base damage : 305 to 357 Arcane damage and then an additional 600 Arcane damage over 12 sec.
    MF_coeff = 0.15
    MF_mana_cost = 450
    if is_T6_2:
        MF_coeff_dot = 0.65
    else:
        MF_coeff_dot = 0.52
    # Starfire base damage : Causes 550 to 647 Arcane damage -> 658 on average
    SF_coeff = 1

    #Paddington - Added spell downranking
    if SF_rank == 6:
        SF_mana_cost = 287
        SF_average_damage = 553.5
    elif SF_rank == 7:
        SF_mana_cost = 309
        SF_average_damage = 614.5
    else:
        SF_mana_cost = 337
        SF_average_damage = 658
        
    MF_average_damage = 397.5
    if is_T6_2:
        MF_average_dot_damage = 720
    else:
        MF_average_dot_damage = 600
    partial_coeff = 0.5 # For the moment, let's say that in average, partials get 50% damage reduction
    sf_cast_time = 3
    sf_cast_time_ng = 2.5
    #Paddington - Added Insect Swarm
    IS_coeff = 0.76
    IS_mana_cost = 175
    IS_average_dot_damage = 792
    
    wrath_coeff = 0.65
    wrath_mana_cost = 232
    wrath_average_damage = 448.5
    wrath_cast_time = 1.5
    wrath_cast_time_ng = 1
    
    
    #Paddington - Added mana potions
    if is_manapot:
        manapot_cd = 0
        manapot_up = True
    else:
        manapot_cd = 60
        manapot_up = False
    
    
    # Apply spell haste coefficients here
    # 15.77 Spell Haste Rating increases casting speed by 1%
    # % Spell Haste at level 70 = (Haste Rating / 15.77)
    # New Casting Time = Base Casting Time / (1 + (% Spell Haste / 100))
    spell_haste = haste / 15.77
    # sf_cast_time = 3 / (1 + (spell_haste/100))
    # sf_cast_time_ng = 2.5 / (1 + (spell_haste/100))
    # print("SF Cast time : " + str(sf_cast_time))
    # print("SF NG Cast time : " + str(sf_cast_time_ng))

    # Spell power calculation for fight SP + lunar guidance 
    if lunar_guidance:
        spellpower = spellpower + 0.25 * intel
    if spellfire:
        spellpower = spellpower + 0.07 * intel
    if is_divine_spirit:
        spellpower = spellpower + .1 * spirit
    
    # Hit chance
    # 12.6 Spell Hit Rating -> 1%
    hit_chance = min(99, 83 + (hit_score/12.6) + balance_of_power)
    hit_chance_percent_value = hit_chance / 100
    logfunc("Hit chance is : " + str(hit_chance))

    # Crit chance
    # At level 70, 22.1 Spell Critical Strike Rating -> 1%
    # Druids receive 1% Spell Critical Strike chance for every 79.4 points of intellect.
    MF_crit_percent = crit_score/22.1 + intel/79.4 + improved_mf + moonkin_form 
    MF_crit_percent_value = MF_crit_percent / 100
    logfunc("Moonfire crit chance is : " + str(MF_crit_percent))
    if is_T6_4 == True:
        SF_crit_percent =  crit_score/22.1 + intel/79.4 +  + moonkin_form + focused_starlight + 5
    else: 
        SF_crit_percent =  crit_score/22.1 + intel/79.4 +  + moonkin_form + focused_starlight
    SF_crit_percent_value = SF_crit_percent / 100
    logfunc("Starfire crit chance is : " + str(SF_crit_percent))
    
    wrath_crit_percent =  crit_score/22.1 + intel/79.4 +  + moonkin_form + focused_starlight 
    logfunc("Wrath crit chance is : " + str(wrath_crit_percent))
    
    logfunc("Spellpower is  : " + str(spellpower))
 
    # Crit coeff
    if csd_equiped:
        crit_coeff = 2.09
    else:
        crit_coeff = 2
    
    # Spellstrike bonus:
    if spellstrike:
        spellstrike_bonus = 92
    else:
        spellstrike_bonus = 0
    
    # Prepare and launch the simulations
    loop_size = 1 # number of fights simulated, Paddington - Changed to 1 and use 2nd cell for number of fights
    average_dps = 0
    n = 0
    while n < loop_size:
        n = n + 1
        # Initialization
        loopcount = 0
        total_damage_done = 0
        damage = 0
        fight_time = 0
        spellstrike_uptime = 0
        eye_of_mag_uptime = 0
        ff_uptime = 0
        mf_uptime = 0
        is_uptime = 0
        trinket_uptime = 0
        trinket_cd_timer = 0
        is_trinket_active = False
        #Paddington - Added on-use trinket internal cooldown
        on_use_icd_timer = 0
        is_trinket_available = True
        is_ff_up = False
        is_mf_up = False
        #Paddington - Added Insect Swarm
        is_is_up = False
        is_ng = False
        is_eye_of_mag_triggered = False
        is_sextant_of_unstable_currents_triggered = False
        is_ashtongue_triggered = False
        spellstrike_proc = False
        ng_proc = False
        drum_time = 0
        drum_cd = 0
        eye_of_quagg_icd = 0
        eye_of_quagg_proc = False
        eye_of_quagg_uptime = 0
        sextant_of_unstable_currents_icd = 0
        sextant_of_unstable_currents_proc = False
        sextant_of_unstable_currents_uptime = 0
        ashtongue_uptime = 0
        skull_uptime = 0
        skull_cd = 0
        skull_active = False
        lust_amount = lust_count
        lust_uptime = 0
        lust_start = 1 - (lust_when/100)
        lusted = False
        fight_haste = spell_haste
        
        #Paddington - Added individual spell damage totals
        SF_damage = 0
        MF_damage = 0
        IS_damage = 0
        Wrath_damage  = 0
        #Treant_damage = 0
        SF_hit = False
        MF_hit = False
        IS_hit = False
        Wrath_hit = False




        # Time to kick ass and chew bubble gum
        while fight_time <= fight_length:
            loop_duration = max(1, (1.5 / (1 + (fight_haste / 100)))) #GCD - can't be less, it's the rule !
            damage = 0
            # adding a variable to keep the initial spellpower to revert to this value in case a trinket bonus
            # fades out before the end of the SF / Wrath cast (ie. no spellpower snapshot)
            loop_start_spellpower = spellpower
            if spellstrike_proc:
                fight_spell_power = spellpower + spellstrike_bonus
            else:
                fight_spell_power = spellpower
           
            if is_eye_of_mag_triggered:
                fight_spell_power = fight_spell_power + 170  
            
            if is_sextant_of_unstable_currents_triggered:
                fight_spell_power = spellpower + 190
 
            if is_ashtongue_triggered:
                fight_spell_power = spellpower + 150
            
            if is_drums and drum_cd <= 0:
                drum_time = 30
                drum_cd = 120
                drums_up = True
                fight_haste = fight_haste + (80/15.77)
               # logfunc(" drums up !!! fight haste is " + str(fight_haste))
                
            if is_lust and lust_amount >= 1 and fight_time >= (lust_start * fight_length) and lusted == False:
                lust_amount = lust_amount - 1
                fight_haste = fight_haste + 30
                lust_uptime = 40
                lusted = True
                #logfunc("lust up, lust amount is " + str(lust_amount) + " fight time is " + str(fight_time))
            #Paddington - Added Dark Iron Smoking Pipe
            if dark_iron_smoking_pipe and is_trinket_activable and is_ff_up:
                fight_spell_power = fight_spell_power + spellpower_trinket_bonus
                on_use_icd_timer = on_use_icd
                trinket_cd_timer = trinket_cd
                is_trinket_activable = False

            if silver_crescent and is_trinket_activable and is_ff_up:
                fight_spell_power = fight_spell_power + spellpower_trinket_bonus
                on_use_icd_timer = on_use_icd
                trinket_cd_timer = trinket_cd
                is_trinket_activable = False
                
            if skull and skull_cd <= 0 and is_ff_up:
                fight_haste = fight_haste + (175/15.77)
                skull_uptime = 20 
                skull_cd = 120
                skull_active = True
                #logfunc("skull activated!!")
            
            #Paddington - Added mana potions
            if manapot_up and mana < 7500:
                mana = mana + (1800 + 3000)/2
                manapot_cd = 60
                manapot_up = False
                logfunc("Mana Potion Used!")
                
                
                
    
            # if FF not up, cast FF
            #Paddington - Added mana management
            if not is_ff_up and mana >= FF_mana_cost:
                logfunc("Casting Faerie Fire !")
                #Paddington - Added mana management
                mana = mana - FF_mana_cost
                logfunc("Current mana:" + str(mana))
                loop_duration = max(1, (1.5 / (1 + (fight_haste/100)))) #GCD
                is_crit = False # can't crit on FF
                damage = 0 # and no damage applied
                if(numpy.random.random() <= hit_chance_percent_value):
                    is_hit = True
                    ff_uptime = 40
                    is_ff_up = True
                    # Test if spellstrike is proc -> Commented out because tested later on in the code
                    # spellstrike_proc = (numpy.random.randint(1, high = 101, size = 1) <= 10)
                else:
                    is_hit = False
                    logfunc("Faerie Fire -> Resist !")
                        
            # if Moonfire not up, cast Moonfire
            else:
                 # Once FF is up trigger trinket if available.
                if is_trinket_active:
                    # apply spell modifier
                    fight_spell_power = fight_spell_power + spellpower_trinket_bonus                    
                else:
                    if is_trinket_available and is_trinket_activable:
                        # activate trinket
                        logfunc("Trinket activation !!")
                        is_trinket_active = True
                        # start the chrono of activation
                        trinket_uptime = trinket_duration
                        # start the chrono of CD
                        trinket_cd_timer = trinket_cd
                #Paddington - Added mana management
                if not is_mf_up and is_MF and mana >= MF_mana_cost:
                    logfunc("Casting Moonfire !")
                    #Paddington - Added mana management
                    mana = mana - MF_mana_cost
                    logfunc("Current mana:" + str(mana))
                    loop_duration = max(1, (1.5 / (1 + (fight_haste/100)))) #GCD because we cast a spell
                    # Is it a hit ?
                    if(numpy.random.random() <= hit_chance_percent_value):
                        is_hit = True
                        MF_hit = True
                        # Is it a crit ?
                        is_crit = (numpy.random.random() <= MF_crit_percent_value)
                        # Is it a partial ?
                        #if(numpy.random.randint(1, high = 101, size = 1) <= hit_chance):
                        #    damage = MF_average_damage + MF_coeff * fight_spell_power * partial_coeff
                        damage = MF_average_damage + MF_coeff * fight_spell_power
                        # Apply damage
                        if is_crit:
                            damage = damage * crit_coeff
                            # DoT :
                        if is_T6_2:
                            damage = damage + MF_average_dot_damage + (MF_coeff_dot * fight_spell_power * min(15, (fight_length - fight_time - 1.5))/15)
                        else:
                            damage = damage + MF_average_dot_damage + (MF_coeff_dot * fight_spell_power * min(12, (fight_length - fight_time - 1.5))/12)

                        # There is a Hit ! update model
                        if is_T6_2:
                            is_mf_up = True
                            mf_uptime = 15
                        else:
                            is_mf_up = True
                            mf_uptime = 12
                    else:
                        is_hit = False
                        logfunc("Moonfire -> Resist ! ")
                #Paddington - Added insect swarm use
                elif not is_is_up and is_IS and mana>= IS_mana_cost:
                    logfunc("Casting Insect Swarm !")
                    mana = mana - IS_mana_cost
                    logfunc("Current mana:" + str(mana))
                    loop_duration = max(1, (1.5 / (1 + (fight_haste/100)))) #GCD because we cast a spell
                    # Is it a hit ?
                    if(numpy.random.random() <= hit_chance_percent_value):
                        is_hit = True
                        #Paddington - Added hit check for individual spell dmg totals
                        IS_hit = True
                        # DoT :
                        damage = damage + IS_average_dot_damage + (IS_coeff * fight_spell_power * min(12, (fight_length - fight_time - 1.5))/12)
                        # There is a Hit ! update model
                        is_is_up = True
                        is_uptime = 12
                    else:
                        is_hit = False
                        logfunc("Insect Swarm -> Resist ! ")
                #Paddington - Added wrath option here   
                #Paddington - Added mana management         
                elif casting_wrath and mana >= wrath_mana_cost:
                    # Cast Wrath
                    logfunc("Casting Wrath !")
                    mana = mana - wrath_mana_cost
                    # Is it a hit ?
                    if(numpy.random.randint(1, high = 101, size = 1) <= hit_chance):
                        is_hit = True
                        #Paddington - Added hit check for individual spell dmg totals
                        Wrath_hit = True
                        # Is it a crit ?
                        is_crit = (numpy.random.randint(1, high = 101, size = 1) <= wrath_crit_percent)
                        # Is it a partial ?
                        if(numpy.random.randint(1, high = 101, size = 1) > hit_chance):
                            logfunc("Partial hit !")
                            damage = (wrath_average_damage + (wrath_coeff * fight_spell_power * wrath_of_cenarius * partial_coeff )) * moonfury
                            # logfunc("Damage done : " + str(damage))
                        else:
                            damage = (wrath_average_damage + (wrath_coeff * fight_spell_power * wrath_of_cenarius )) * moonfury
                            logfunc("Damage done : " + str(damage))
                        if is_crit:
                            damage = damage * crit_coeff
                    else:
                        is_hit = False
                        logfunc("Wrath -> Resist ! ")
                    if is_ng:
                        loop_duration = wrath_cast_time_ng
                    else:
                        loop_duration = wrath_cast_time
                    is_ng = False # Consume NG once wrath is cast

                #Paddington - Added mana management
                elif casting_SF and mana >= SF_mana_cost:
                    # Cast Starfire
                    logfunc("Casting Starfire !")
                    #Paddington - Added mana management
                    mana = mana - SF_mana_cost
                    logfunc("Current mana:" + str(mana))
                    sf_cast_time = 3 / (1 + (fight_haste/100))
                    sf_cast_time_ng = 2.5 / (1 + (fight_haste/100))
                    # Computing loop duration
                    if is_ng:
                        sf_cast_time_ng = max(1, (2.5 / (1 + (fight_haste/100))))
                        loop_duration = max(1, sf_cast_time_ng)
                    else:
                        sf_cast_time = max(1, (3 / (1 + (fight_haste/100))))
                        loop_duration = max(1, sf_cast_time)
                    is_ng = False # Consume NG once SF is cast   
                    # Is it a hit ?
                    # if(numpy.random.randint(1, high = 101, size = 1) <= hit_chance):
                    if(numpy.random.random() <= hit_chance_percent_value):
                        is_hit = True
                        #Paddington - Added hit check for individual spell dmg totals
                        SF_hit = True
                        if ashtongue_talisman and numpy.random.randint(1, high = 5, size = 1) == 1:
                            is_ashtongue_triggered = True
                            ashtongue_uptime = 8
                            logfunc("Ashtongue Proc !!")
                        # Is it a crit ?
                        is_crit = (numpy.random.random() <= SF_crit_percent_value)
                        if is_crit:
                            logfunc("Starfire -> Crit ! ")
                        # Is it a partial ?
                        #if(numpy.random.randint(1, high = 101, size = 1) > hit_chance):
                        #    logfunc("Partial hit !")
                        #    damage = (SF_average_damage + (SF_coeff * fight_spell_power * wrath_of_cenarius * partial_coeff )) * moonfury
                            # logfunc("Damage done : " + str(damage))



                        # Let's check if the cast is finished before trinket bonus, if not, we remove the relevant bonus :
                        if spellstrike_proc and (spellstrike_uptime < loop_duration):
                            fight_spell_power = fight_spell_power - spellstrike_bonus 
                            logfunc("Spellstrike fades out to early ...")
                        if is_eye_of_mag_triggered and (eye_of_mag_uptime < loop_duration):
                            fight_spell_power = fight_spell_power - 170
                            logfunc("Eye of Mag fades out to early ...")
                        if is_sextant_of_unstable_currents_triggered and (sextant_of_unstable_currents_uptime < loop_duration):
                            fight_spell_power = fight_spell_power - 190
                            logfunc("Sextant fades out to early ...")
                        if is_ashtongue_triggered and (ashtongue_uptime < loop_duration):
                            fight_spell_power = fight_spell_power - 150
                            logfunc("Ashtongue fades out to early ...")

                        # end of trinket verification
                        #Paddington - Added check for Insect Swarm
                        if is_T5 and is_ng and mf_uptime >= sf_cast_time_ng and is_uptime >= sf_cast_time_ng:
                            damage = (SF_average_damage + (SF_coeff * fight_spell_power * wrath_of_cenarius )) * moonfury * 1.1                            
                        elif is_T5 and mf_uptime >= sf_cast_time:
                            damage = (SF_average_damage + (SF_coeff * fight_spell_power * wrath_of_cenarius )) * moonfury * 1.1                        
                        elif is_T5 and is_uptime >= sf_cast_time:
                            damage = (SF_average_damage + (SF_coeff * fight_spell_power * wrath_of_cenarius )) * moonfury * 1.1
                        else:
                            damage = (SF_average_damage + (SF_coeff * fight_spell_power * wrath_of_cenarius )) * moonfury                            
                        if is_crit:
                            damage = damage * crit_coeff
                            logfunc("Damage done : " + str(damage))
                    else:
                        is_hit = False
                        logfunc("Starfire -> Resist ! ")

                    
                    #Treant_damage = 0
                    #Individual spell damage totals
                    if SF_hit:
                        SF_damage = SF_damage + damage
                        SF_hit = False

                    if MF_hit:
                        MF_damage = MF_damage + damage
                        MF_hit = False

                    if IS_hit:
                        IS_damage = IS_damage + damage
                        IS_hit = False
  
                    if Wrath_hit:
                        Wrath_damage = Wrath_damage + damage
                        Wrath_hit = False
                    

            # Update time and model
            fight_time = fight_time + loop_duration
            ff_uptime = ff_uptime - loop_duration
            mf_uptime = mf_uptime - loop_duration
            trinket_uptime = trinket_uptime - loop_duration
            trinket_cd_timer = trinket_cd_timer - loop_duration
            #Paddington - Added Internal CD for on-use trinkets
            on_use_icd_timer = on_use_icd_timer - loop_duration
            eye_of_mag_uptime = eye_of_mag_uptime - loop_duration
            spellstrike_uptime = spellstrike_uptime - loop_duration
            eye_of_quagg_icd = eye_of_quagg_icd - loop_duration
            eye_of_quagg_uptime = eye_of_quagg_uptime - loop_duration
            sextant_of_unstable_currents_icd = sextant_of_unstable_currents_icd - loop_duration
            sextant_of_unstable_currents_uptime = sextant_of_unstable_currents_uptime - loop_duration
            ashtongue_uptime = ashtongue_uptime - loop_duration
            drum_time = drum_time - loop_duration
            drum_cd = drum_cd - loop_duration
            #####################
            #Paddington - Added Mana management
            manapot_cd = manapot_cd - loop_duration
            mp5_tick = mp5_tick - loop_duration
            ######################
            lust_uptime = lust_uptime - loop_duration
            skull_uptime = skull_uptime - loop_duration
            skull_cd = skull_cd - loop_duration
            # Check the timer on buffs / debuffs                
            if spellstrike_uptime <= 0:
                spellstrike_proc = False
            
            if mf_uptime <= 0:
                is_mf_up = False
            if ff_uptime <= 0:
                is_ff_up = False
            # Trinket 
            if trinket_uptime <= 0:
                is_trinket_active = False
            else:
                is_trinket_active = True
            #Paddington - Added check for trinket internal CD if using 2 on-use trinkets
            if trinket_cd_timer <= 0 and on_use_icd_timer <= 0:
                is_trinket_available = True
            else:
                is_trinket_available = False

            if eye_of_quagg_uptime <= 0 and eye_of_quagg_proc:
                fight_haste = fight_haste - (320/15.77)
                eye_of_quagg_proc = False 
                logfunc("eye of quagg fades. fight haste is " + str(fight_haste))
                
            if eye_of_mag_uptime <= 0:
                is_eye_of_mag_triggered = False

            if sextant_of_unstable_currents_uptime <= 0:
                is_sextant_of_unstable_currents_triggered = False

            if ashtongue_uptime <= 0:
                is_ashtongue_triggered = False
                
            if is_drums and drum_time <= 0 and drums_up:
                fight_haste = fight_haste - (80/15.77)
                drums_up = False
                #logfunc("drums down!!! fight haste is: " + str(fight_haste))
                
            #Paddington - Added mana potion use
            if manapot_cd <= 0:
                manapot_up = True
                logfunc("Mana Potion Available!")
            else:
                logfunc("Mana Potion Down!")
            #Paddington - Added mana management
            if mp5_tick <= 0:
                mana = mana + casting_mp5
                
            if lusted and lust_uptime <= 0:
                fight_haste = fight_haste - 30
                lusted = False
                #logfunc("lust down!")
                
            if skull_active and skull_uptime <= 0:
                fight_haste = fight_haste - (175/15.77)
                skull_active = False
                #logfunc("skull deactivated !!")
                
            # Update nature's grace
            if is_crit:
                if sextant_of_unstable_currents and numpy.random.randint(1, high = 6, size = 1) == 5 and sextant_of_unstable_currents_icd <= 0:
                    is_sextant_of_unstable_currents_triggered = True
                    sextant_of_unstable_currents_uptime = 15
                    sextant_of_unstable_currents_icd = sextant_icd
                    logfunc("Sextant proc!")
                is_ng = True

            total_damage_done = total_damage_done + damage * curse_of_elements * misery

            # If there is a Hit, Check if spellstrike / Quag'eye is proc or refreshed :
            if is_hit: 
                if is_spellstrike and numpy.random.randint(1, high = 11, size = 1) == 10:
                    spellstrike_proc = True
                    spellstrike_uptime = 10
                    logfunc("Spellstrike proc !!!")
                if quagmirran and numpy.random.randint(1, high = 11, size = 1) == 10 and eye_of_quagg_icd <= 0:
                    eye_of_quagg_proc = True
                    eye_of_quagg_uptime = 6
                    eye_of_quagg_icd = trinket_icd
                    fight_haste = fight_haste + (320/15.77)
                    logfunc("Eye of Quagmirran proc !!!, fight haste is: "+ str(fight_haste)) 
                    
            # If there is a resist, check eye of mag proc :
            if eye_of_mag and not is_hit:
                is_eye_of_mag_triggered = True
                eye_of_mag_uptime = 10
                logfunc("Eye of mag proc !!!")

            # Print output
            logfunc("Loop Duration : " + str(loop_duration))
            logfunc("Loop Damage : " + str(damage))
            if damage > 0:
                loopcount = loopcount + 1
            
        #print("damage loops: ", loopcount)  Used to track how many loops actually do damage in a fight i.e. before mana runs out.
        logfunc("Overall damage done : " + str(total_damage_done))
        #Paddington - Added individual spell damage totals
        #print("Starfire damage : ", str(SF_damage))
        #print("Starfall DPS: ", SF_damage/fight_time)
        #print("Percent Starfall damage: ", (SF_damage/total_damage_done)*100)
        #print("Wrath damage : ", str(Wrath_damage))
        #print("Wrath DPS: ", Wrath_damage/fight_time)
        #print("Percent Wrath damage: ", (Wrath_damage/total_damage_done)*100)
        #print("Moonfire damage : ", str(MF_damage))
        #print("Moonfire DPS: ", MF_damage/fight_time)
        #print("Percent Moonfire damage: ", (MF_damage/total_damage_done)*100)
        #print("Insect Swarm damage : ", str(IS_damage))
        #print("Insect Swarm DPS: ", IS_damage/fight_time)
        #print("Percent Insect Swarm damage: ", (IS_damage/total_damage_done)*100)
        logfunc("Overall DPS : "  + str(total_damage_done/fight_time)) # We use fight_time here in case SF lands after the fight_length mark
        average_dps = average_dps + (total_damage_done/fight_time)
        break
    
    #Paddington - Changed to average dps. Seems more accurate.
    #real_average_dps = average_dps / loop_size
    #logfunc("Average DPS : " + str(real_average_dps))
    return(average_dps)
    #return(real_average_dps)
 
 
# Adapting the function to panda dataframes
#v_compute_dps = numpy.vectorize(compute_dps)

In [28]:
# Run this part to calculate the average dps value of your configuration
n = 0
all_dps = 0
loop_size = 100
damage = 0

while n < loop_size :
    n = n + 1
    # all_dps = all_dps + compute_dps(intel = 381, crit_score = 243, hit_score = 135, spellpower = 1162, haste = 0)
    all_dps = all_dps + compute_dps(mana, intel, crit_score, hit_score, spellpower, haste,
                                    is_csd, is_spellstrike, is_spellfire, is_T5, is_T6_2, is_T6_4, is_manapot,
                                    is_MF, is_IS, SF_rank, casting_wrath, casting_SF)
average_dps = all_dps / loop_size
print("Average DPS with current configuration is : ", average_dps)


Average DPS with current configuration is :  938.0512338946414
